# 계층형 분류기와 단일 분류기 성능 비교

동일한 데이터셋에 대해 학습한 계층형 분류 방식과 단일 분류 방식의 성능을 비교하고 분석합니다. 정확도, 정밀도, 재현율 등 다양한 평가지표를 사용하여 성능을 평가하고 결과를 비교합니다.

In [28]:
import pandas as pd
import numpy as np
import re

FILE_PATH = 'assets/' 
MODEL_PATH = 'models/'

# 서브레딧 및 그룹 정의
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries',
    'learnprogramming', 'cscareerquestions', 'SideProject',
    'TrueFilm', 'booksuggestions', 'TrueGaming'
]
GROUP_MAP = {'Mystery': SUBREDDITS[0:3], 'Dev': SUBREDDITS[3:6], 'Culture': SUBREDDITS[6:9]}

VECTOR_DIMENSION = 5000

In [29]:
# 데이터 로드
df = pd.read_csv(FILE_PATH + 'reddit_posts_preprocessed.csv')

# 테스트 데이터 확인
df.head()

,subreddit,preprocessed_content
0,Thetruthishere,truth leviathan leviathan isnt scary monster a...
1,Thetruthishere,streetlight front house turns every time walk ...
2,Thetruthishere,reflection window seconds behind nearly empty ...
3,Thetruthishere,strange knocking empty apartment night ive liv...
4,Thetruthishere,whats unexplainable thing youve ever witnessed...


## 모델 로드

In [30]:
# 모델 로드
import joblib

# 상위 분류기 로드
top_classifier = joblib.load(MODEL_PATH + 'top_classifier.pkl')

# 하위 분류기 로드
sub_classifiers = {}
for group_name in GROUP_MAP.keys():
    sub_classifiers[group_name] = joblib.load(MODEL_PATH + f'sub_classifier_{group_name}.pkl')

# 단일 분류기 로드
single_classifier = joblib.load(MODEL_PATH + 'single_classifier.pkl')

## 성능 비교

In [31]:
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# 계층형 분류기 성능 평가
def evaluate_hierarchical_classifier(X_test_vec, y_test_top, y_test_sub):
    # 상위 분류기 예측
    y_pred_top = top_classifier.predict(X_test_vec)
    
    # 하위 분류기 예측
    y_pred_sub = []
    for i, group in enumerate(y_pred_top):
        # 예측된 그룹에 맞는 하위 분류기 선택
        sub_clf = sub_classifiers[group]
        
        # X_test_vec[i]는 1차원 배열이므로, (1, -1)로 reshape하여 2차원으로 만들어 예측
        # (참고: X_test_vec이 numpy array인 경우 reshape 필요)
        sub_pred = sub_clf.predict(X_test_vec[i].reshape(1, -1))
        y_pred_sub.append(sub_pred[0])
    
    # 성능 평가
    print("="*50)
    print("[계층형 분류기 성능 평가]")
    print(classification_report(y_test_sub, y_pred_sub))
    print("="*50)

# 단일 분류기 성능 평가
def evaluate_single_classifier(X_test_vec, y_test):
    # 단일 분류기 예측
    y_pred = single_classifier.predict(X_test_vec)
    
    # 성능 평가
    print("="*50)
    print("[단일 분류기 성능 평가]")
    print(classification_report(y_test, y_pred))
    print("="*50)

# 텍스트 벡터화
vectorizer = TfidfVectorizer(max_features=VECTOR_DIMENSION, ngram_range=(1, 2))
X_test_vec = vectorizer.fit_transform(df['preprocessed_content']).toarray()

# 정답 레이블 준비
reverse_group_map = {subreddit: group for group, subreddits in GROUP_MAP.items() for subreddit in subreddits}
y_group = df['subreddit'].map(reverse_group_map).tolist()
y_test_sub = df['subreddit'].tolist()

# 평가 함수 실행

# 계층형 분류기 평가 (벡터화된 X_test_vec 전달)
evaluate_hierarchical_classifier(X_test_vec, y_group, y_test_sub)

# 단일 분류기 평가 (벡터화된 X_test_vec 전달)
evaluate_single_classifier(X_test_vec, y_test_sub)

[계층형 분류기 성능 평가]
                      precision    recall  f1-score   support

Glitch_in_the_Matrix       0.94      0.98      0.96       987
         SideProject       0.93      0.96      0.95       946
      Thetruthishere       0.97      0.93      0.95       950
            TrueFilm       0.99      0.98      0.98       988
          TrueGaming       0.98      0.99      0.99       989
 UnresolvedMysteries       0.99      0.99      0.99       991
     booksuggestions       0.99      0.99      0.99       999
   cscareerquestions       0.91      0.90      0.91       987
    learnprogramming       0.90      0.90      0.90       995

            accuracy                           0.96      8832
           macro avg       0.96      0.96      0.96      8832
        weighted avg       0.96      0.96      0.96      8832

[단일 분류기 성능 평가]
                      precision    recall  f1-score   support

Glitch_in_the_Matrix       0.95      0.90      0.92       987
         SideProject       0.93    